In [38]:
import numpy as np 
import yaml 
import pandas as pd

from pathlib import Path
import yaml 
from copy import deepcopy

import pickle


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



# Make 50Hz versions of the 10 feature-gain architectures

Alter front end config to apply 50Hz limit on phase locking

In [ ]:
## write configs 


## import default config 
outdir = Path("config/arch_search")
outdir.mkdir(exist_ok=True, parents=True)

base_config = yaml.load(open("config/binaural_attn/word_task_half_co_loc_v09_50Hz_cutoff.yaml", 'r'), Loader=yaml.FullLoader)
audio_config_50Hz = deepcopy(base_config['audio'])

In [20]:
### Get alternate configs 
arch_configs = pd.read_pickle('binaural_train_manifests/v10_alt_arch_search_manifest.pkl')[:-1] # -1 to skip arch 13. arch 13 didn't optimize  
arch_configs += [Path("config/binaural_attn/word_task_v10_main_feature_gain_config.yaml")]
len(arch_configs)
arch_configs

[PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_1.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_2.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_4.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_6.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_7.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_8.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_9.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_10.yaml'),
 PosixPath('config/arch_search/word_task_v10_4MGB_ln_first_arch_12.yaml'),
 PosixPath('config/binaural_attn/word_task_v10_main_feature_gain_config.yaml')]

# Write configs out

### Update existing configs with name and new audio frontend config 

In [35]:
# Convert architectures in archs to configs
from copy import deepcopy
from pprint import pprint

## import default config 
outdir = Path("config/v10_50Hz_archs")
outdir.mkdir(exist_ok=True, parents=True)

config_dict = {}


for ix, base_config in enumerate(arch_configs):
    arch_config = yaml.load(open(base_config, 'r'), Loader=yaml.FullLoader)
    arch_config = deepcopy(arch_config)
    arch_name = base_config.stem
    # update path audio config 
    model_name = f"{arch_name}_50Hz"
    arch_config['audio'] = audio_config_50Hz 
    arch_config['model_name'] = f"{arch_name}_50Hz"
    config_name = outdir / f"{model_name}.yaml"
    # print(config_name)
    # print(arch_config)
    config_dict[ix] = config_name.resolve()
    # # break
    # with open(config_name, 'w') as f:
    #     yaml.dump(arch_config, f, default_flow_style=False)
pprint(config_dict)

{0: PosixPath('/net/vast-storage/scratch/vast/mcdermott/imgriff/projects/torch_2_aud_attn/config/v10_50Hz_archs/word_task_v10_4MGB_ln_first_arch_1_50Hz.yaml'),
 1: PosixPath('/net/vast-storage/scratch/vast/mcdermott/imgriff/projects/torch_2_aud_attn/config/v10_50Hz_archs/word_task_v10_4MGB_ln_first_arch_2_50Hz.yaml'),
 2: PosixPath('/net/vast-storage/scratch/vast/mcdermott/imgriff/projects/torch_2_aud_attn/config/v10_50Hz_archs/word_task_v10_4MGB_ln_first_arch_4_50Hz.yaml'),
 3: PosixPath('/net/vast-storage/scratch/vast/mcdermott/imgriff/projects/torch_2_aud_attn/config/v10_50Hz_archs/word_task_v10_4MGB_ln_first_arch_6_50Hz.yaml'),
 4: PosixPath('/net/vast-storage/scratch/vast/mcdermott/imgriff/projects/torch_2_aud_attn/config/v10_50Hz_archs/word_task_v10_4MGB_ln_first_arch_7_50Hz.yaml'),
 5: PosixPath('/net/vast-storage/scratch/vast/mcdermott/imgriff/projects/torch_2_aud_attn/config/v10_50Hz_archs/word_task_v10_4MGB_ln_first_arch_8_50Hz.yaml'),
 6: PosixPath('/net/vast-storage/scratch

In [39]:
manifest_path = 'binaural_train_manifests/v10_50Hz_model_manifest.pkl'

with open(manifest_path, 'wb') as f:
    pickle.dump(config_dict, f)

In [44]:
Path(manifest_path).resolve()

PosixPath('/net/vast-storage/scratch/vast/mcdermott/imgriff/projects/torch_2_aud_attn/binaural_train_manifests/v10_50Hz_model_manifest.pkl')

## Make sure configs are compat with model 

In [42]:
from src.spatial_attn_lightning import BinauralAttentionModule
import yaml 

In [43]:
for config_path in config_dict.values():
    config = yaml.load(open(config_path, 'r'), Loader=yaml.FullLoader)
    n_params = 0
    for param in BinauralAttentionModule(config).model.parameters():
        n_params += param.numel()
    print(f"{n_params/1e6:.2f} M")

Using explicit dim specification for demeaning in audio transforms
Using BinauralAuditoryAttentionCNN
v08 True
num_classes={'num_words': 800}
Model performing word task
Using singe gain function per layer
Conv block order: LN -> Conv -> ReLU
fc_attn: True
coch_affine: True
Using dataset BinauralAttentionDataset
BinauralAuditoryAttentionCNN(
  (model_dict): ModuleDict(
    (norm_coch_rep): LayerNorm((2, 40, 20000), eps=1e-05, elementwise_affine=True)
    (attn0): SimpleAttentionalGain()
    (conv_block_0): Sequential(
      (0): LayerNorm((2, 40, 20000), eps=1e-05, elementwise_affine=True)
      (1): Conv2d(2, 32, kernel_size=(1, 59), stride=(1, 1), bias=False)
      (2): ReLU()
    )
    (hann_pool_0): HannPooling2d()
    (attn1): SimpleAttentionalGain()
    (conv_block_1): Sequential(
      (0): LayerNorm((32, 21, 19942), eps=1e-05, elementwise_affine=True)
      (1): Conv2d(32, 64, kernel_size=(2, 72), stride=(1, 1), bias=False)
      (2): ReLU()
    )
    (hann_pool_1): HannPooling2